In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
#Implementing Standard Scalar
from sklearn.preprocessing import StandardScaler

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#All 4 files
#df_constPost = pd.read_csv('Anwar CSV_Const Post_EDA\Merged ConstPost Dataset.csv')

#1416 files ConstPost Dataset
#df_constPost = pd.read_csv('Merged 1416 ConstPost Dataset.csv')

#1416 Constant Speed Offset
#df_constPost = pd.read_csv("/content/drive/MyDrive/Speed Offset Dataset/RandomSpeedOffset_final.csv")
df_constPost = pd.read_csv("/content/drive/MyDrive/Speed Offset Dataset/ConstantSpeedOffset_final.csv")
print(df_constPost.shape)

(820097, 34)


In [ ]:
df_constPost['attack?'].value_counts()

0    572039
1    248058
Name: attack?, dtype: int64

In [ ]:
df_constPost.columns

Index(['type', 'rcvTime', 'sendTime', 'sender', 'senderPseudo', 'messageID',
       'file_name', 'density_level', 'pos_x', 'pos_y', 'pos_z', 'pos_noise_x',
       'pos_noise_y', 'pos_noise_z', 'spd_x', 'spd_y', 'spd_z', 'spd_noise_x',
       'spd_noise_y', 'spd_noise_z', 'acl_x', 'acl_y', 'acl_z', 'acl_noise_x',
       'acl_noise_y', 'acl_noise_z', 'hed_x', 'hed_y', 'hed_z', 'hed_noise_x',
       'hed_noise_y', 'hed_noise_z', 'attack?', 'receiver_id'],
      dtype='object')

In [ ]:
#Version 1: All features
# df_constPost.drop(['type','file_name','pos_z','pos_noise_z','spd_z','spd_noise_z',
#                   'acl_z','acl_noise_z','hed_z','hed_noise_z',
#                   'pos_noise_x','pos_noise_y','spd_noise_x','spd_noise_y','acl_noise_x','acl_noise_y',
#                  'hed_noise_x','hed_noise_y','density_level'],axis=1,inplace=True)

#Version 2 (Without RcvTime, SendTime, RcvID, SenderID, SendPseudo, MessageID)
df_constPost.drop(['type','file_name','pos_z','pos_noise_z','spd_z','spd_noise_z',
                  'acl_z','acl_noise_z','hed_z','hed_noise_z',
                  'pos_noise_x','pos_noise_y','spd_noise_x','spd_noise_y','acl_noise_x','acl_noise_y',
                 'hed_noise_x','hed_noise_y','density_level', 
                  'rcvTime', 'sendTime', 'sender', 'senderPseudo', 'messageID','receiver_id'],axis=1,inplace=True)

#Version 3 (Without SendTime, RcvID, SenderID, SendPseudo, MessageID) [Diff. between V2 and 3 is RcvTime]
# df_constPost.drop(['type','file_name','pos_z','pos_noise_z','spd_z','spd_noise_z',
#                   'acl_z','acl_noise_z','hed_z','hed_noise_z',
#                   'pos_noise_x','pos_noise_y','spd_noise_x','spd_noise_y','acl_noise_x','acl_noise_y',
#                  'hed_noise_x','hed_noise_y','density_level', 
#                               'sendTime', 'sender', 'senderPseudo', 'messageID','receiver_id'],axis=1,inplace=True)

In [ ]:
df_constPost.columns

Index(['pos_x', 'pos_y', 'spd_x', 'spd_y', 'acl_x', 'acl_y', 'hed_x', 'hed_y',
       'attack?'],
      dtype='object')

In [ ]:
# getting number of rows
#Splitting

row_count = df_constPost.shape[0]
#print(row_count)

row_train = round(row_count*0.8)
row_test = round(row_count-row_train)

print(row_train)
print(row_test)

656078
164019


In [ ]:
df_train = df_constPost.head(row_train)
print(df_train.shape)

df_test = df_constPost.tail(row_test)
print(df_test.shape)

(656078, 9)
(164019, 9)


In [ ]:
#Train test split

#TRAINING

#Version 1
#X_train = df_train[['spd_x', 'spd_y','hed_x','hed_y','pos_x', 'pos_y', 'acl_x','acl_y', 'rcvTime', 'sendTime', 'receiver_id', 'sender','senderPseudo','messageID']]

#Version 2
X_train = df_train[['spd_x', 'spd_y','hed_x','hed_y','pos_x', 'pos_y', 'acl_x','acl_y']]

#Version 3
#X_train = df_train[['spd_x', 'spd_y','hed_x','hed_y','pos_x', 'pos_y', 'acl_x','acl_y', 'rcvTime']]

#Common for all 3
y_train = df_train['attack?']

#TESTING

#Version 1
#X_test = df_test[['spd_x', 'spd_y','hed_x','hed_y','pos_x', 'pos_y', 'acl_x','acl_y', 'rcvTime', 'sendTime', 'receiver_id', 'sender','senderPseudo','messageID']]

#Version 2
X_test = df_test[['spd_x', 'spd_y','hed_x','hed_y','pos_x', 'pos_y', 'acl_x','acl_y']]

#Version 3
#X_test = df_test[['spd_x', 'spd_y','hed_x','hed_y','pos_x', 'pos_y', 'acl_x','acl_y', 'rcvTime']]

#Common for all 3
y_test = df_test['attack?']

# print(y_test.value_counts())

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((656078, 8), (164019, 8), (656078,), (164019,))

In [ ]:
# data normalization with sklearn

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train = norm.transform(X_train)

# transform testing dataabs
X_test = norm.transform(X_test)

#Implementing Standard Scalar

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=500, activation='relu', hidden_layer_sizes=(45, 30, 15), batch_size=128)
mlp.fit(X_train,y_train)

#Time calc for train
start_time_train = time.time()
#Train Accuracy
y_train_mlp = mlp.predict(X_train)
print('Accuracy Train mlp for :', accuracy_score(y_train, y_train_mlp)*100)

end_time_train = time.time()
execution_time_train = end_time_train - start_time_train
print('Execution Time for train:', execution_time_train)

#Time calc for test
start_time_test = time.time()

#Test Accuracy
# make predictions for test data
y_pred_mlp = mlp.predict(X_test)

# evaluate predictions
print('Accuracy Test mlp:', accuracy_score(y_test, y_pred_mlp)*100)

#Evaluating the XG algorithm
print(confusion_matrix(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))

end_time_test = time.time()
execution_time_test = end_time_test - start_time_test
print('Execution Time for test:', execution_time_test)

Accuracy Train mlp for : 99.62230100689247
Execution Time for train: 1.249405860900879
Accuracy Test mlp: 99.44884434120436
[[115228    127]
 [   777  47887]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    115355
           1       1.00      0.98      0.99     48664

    accuracy                           0.99    164019
   macro avg       1.00      0.99      0.99    164019
weighted avg       0.99      0.99      0.99    164019

Execution Time for test: 0.7313210964202881


In [34]:
#MLP score with Cross validation
start_time_test = time.time()

mlp_Cross = cross_val_score(mlp, X_train, y_train, cv = 5)
mlp_Cross_mean = mlp_Cross.mean()
print('Mean_Accuracy Kfold mlp:', mlp_Cross_mean*100)

end_time_test = time.time()
execution_time_test = end_time_test - start_time_test
print('Execution Time for mlp Cross Val:', execution_time_test)

Mean_Accuracy Kfold mlp: 98.98853259331014
Execution Time for mlp Cross Val: 2080.7456271648407


In [ ]:
# pred = mlp.predict(X_test)
# pred

# print('Accuracy Test MLP:', accuracy_score(y_test, pred)*100)

# #Evaluating the XG algorithm
# print(confusion_matrix(y_test, pred))
# print(classification_report(y_test, pred))

Accuracy Test MLP: 98.64649827154172
[[114428    927]
 [  1293  47371]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    115355
           1       0.98      0.97      0.98     48664

    accuracy                           0.99    164019
   macro avg       0.98      0.98      0.98    164019
weighted avg       0.99      0.99      0.99    164019



In [ ]:
#HyperParameter Tuning Start
# parameter_space = {
#     'hidden_layer_sizes': [(20), (40,20), (45,30,15)],
#     'activation': ['tanh', 'relu'],
#     'solver': ['sgd', 'adam'],
#     'alpha': [0.0001, 0.05],
#     'learning_rate': ['constant','adaptive'],
# }


In [ ]:
# layers = [[20], [40, 20], [45, 30, 15]]
# activations = ['tanh', 'relu']
# solver= ['sgd', 'adam']
# learning_rate= ['constant','adaptive']
# parameter_space = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30], solver=solver, learning_rate=learning_rate)
# #grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [ ]:
# from sklearn.model_selection import GridSearchCV

# clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
# clf.fit(X_train, y_train)